In [ ]:
import cv2
import numpy as np
import copy as cp

diskStrelKernel = np.array([[0,1,0],[1,1,1],[0,1,1]],dtype=np.uint8)

def MidpointCircle(i, radius, xc, yc, value):

    x = 0;
    y = (radius);
    d = (1 - radius);
    i[xc, yc+y] = value;
    i[xc, yc-y] = value;
    i[xc+y, yc] = value;
    i[xc-y, yc] = value;
    
    while ( x < y - 1 ):
        x = x + 1;
        if ( d < 0 ):
            d = d + x + x + 1;
        else :
            y = y - 1;
            a = x - y + 1;
            d = d + a + a;
        i[ x+xc,  y+yc] = value;
        i[ y+xc,  x+yc] = value;
        i[ y+xc, -x+yc] = value;
        i[ x+xc, -y+yc] = value;
        i[-x+xc, -y+yc] = value;
        i[-y+xc, -x+yc] = value;
        i[-y+xc,  x+yc] = value;
        i[-x+xc,  y+yc] = value;
    return i
        
def DaugmanPatternFullCircle(ringWidth,diskSize):
    ringTemplate =[]
    for i in xrange(0,len(diskSize)):
        ring = np.zeros((diskSize[i]*2+1,diskSize[i]*2+1),dtype=np.uint8)
        for j in xrange(1,ringWidth+1):
            ring = MidpointCircle(ring,diskSize[i]-j+1,diskSize[i],diskSize[i],1)
        ring = cv2.dilate(ring,diskStrelKernel)
        ring = np.float32(ring)
        ring = ring/sum(sum(ring))
        ringTemplate.append(ring)
    return ringTemplate
    
def DaugmanPatternTopHalf(ringWidth,diskSize):
    ringTemplate =[]
    for i in xrange(0,len(diskSize)):
        ring = np.zeros((diskSize[i]*2+1,diskSize[i]*2+1),dtype=np.uint8)
        for j in xrange(1,ringWidth+1):
            ring = MidpointCircle(ring,diskSize[i]-j+1,diskSize[i],diskSize[i],1)
        ring = cv2.dilate(ring,diskStrelKernel)
        ring[0:round(ring.shape[0]/2),:] = 0
        ring = np.float32(ring)
        ring = ring/sum(sum(ring))
        ringTemplate.append(ring)
    return ringTemplate

#(1,[ 24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.])
#？这个函数是在做什么
def DaugmanPatternBottomHalf(ringWidth,diskSize):
    ringTemplate =[]
    #xrange是生成器，使用list(xrange(X,X,X))可以生成序列
    #print '&'*40
    #print xrange(0,len(diskSize))->>xrange(13)
    #print list(xrange(0,len(diskSize)))
    for i in xrange(0,len(diskSize)):
        #print 'i=%d\n'%(i)
        ring = np.zeros((diskSize[i]*2+1,diskSize[i]*2+1),dtype=np.uint8)
        #print ring
        for j in xrange(1,ringWidth+1):
            ring = MidpointCircle(ring,diskSize[i]-j+1,diskSize[i],diskSize[i],1)
        ring = cv2.dilate(ring,diskStrelKernel)
        #print ring
        ring[round(ring.shape[0]/2):,:] = 0
        ring = np.float32(ring)
        #print ring
        ring = ring/sum(sum(ring))
        #print ring
        ringTemplate.append(ring)
    #print '&'*40
    return ringTemplate
  
#(1,[ 24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.])
#？这个函数是在做什么    
def DaugmanPatternSymmetryBrace(ringWidth,diskSize):
    ringTemplate =[]
    for i in xrange(0,len(diskSize)):
        ring = np.zeros((diskSize[i]*2+1,diskSize[i]*2+1),dtype=np.uint8)
        for j in xrange(1,ringWidth+1):
            ring = MidpointCircle(ring,diskSize[i]-j+1,diskSize[i],diskSize[i],1)
        #print ring
        #cv2.imshow('ygtest',ring*255)
        #cv2.waitKey()
        ring = cv2.dilate(ring,diskStrelKernel)
        ring[round(ring.shape[0]/5*4):,:] = 0
        ring[0:round(ring.shape[0]/5):,:] = 0
        ring = np.float32(ring)
        ring = ring/sum(sum(ring))
        ringTemplate.append(ring)
    return ringTemplate
    
def DaugmanPatternLeftBrace(ringWidth,diskSize):
    ringTemplate =[]
    for i in xrange(0,len(diskSize)):
        ring = np.zeros((diskSize[i]*2+1,diskSize[i]*2+1),dtype=np.uint8)
        for j in xrange(1,ringWidth+1):
            ring = MidpointCircle(ring,diskSize[i]-j+1,diskSize[i],diskSize[i],1)
        ring = cv2.dilate(ring,diskStrelKernel)
        ring[round(ring.shape[0]/5*4):,:] = 0
        ring[0:round(ring.shape[0]/5):,:] = 0
        ring[:,round(ring.shape[1]/2):] = 0
        ring = np.float32(ring)
        ring = ring/sum(sum(ring))
        ringTemplate.append(ring)
    return ringTemplate
    
def DaugmanPatternRightBrace(ringWidth,diskSize):
    ringTemplate =[]
    for i in xrange(0,len(diskSize)):
        ring = np.zeros((diskSize[i]*2+1,diskSize[i]*2+1),dtype=np.uint8)
        for j in xrange(1,ringWidth+1):
            ring = MidpointCircle(ring,diskSize[i]-j+1,diskSize[i],diskSize[i],1)
        ring = cv2.dilate(ring,diskStrelKernel)
        ring[round(ring.shape[0]/5*4):,:] = 0
        ring[0:round(ring.shape[0]/5):,:] = 0
        ring[:,0:round(ring.shape[1]/2)] = 0
        ring = np.float32(ring)
        ring = ring/sum(sum(ring))
        ringTemplate.append(ring)
    return ringTemplate

#输入(1,[ 24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.])
def GenerateDaugmanTemplateSet(ringWidth, diskSize):
    weights = [];
    weights.append(0.25)
    weights.append(0.40)
    funcSet = [];
    funcSet.append(DaugmanPatternBottomHalf)
    funcSet.append(DaugmanPatternSymmetryBrace)
    results = [GenerateDaugmanTemplateSet.pool.apply_async(f,(ringWidth,diskSize)) for f in funcSet]
    parallelResults = [r.get() for r in results]
    return parallelResults,weights


#myFunction
def GenerateDaugmanTemplateSet2(ringWidth, diskSize):
    parallelResults2,index2=allKindsOfPattern(ringWidth, diskSize)
    #权重还未想好怎么处理，先按照同权重处理
    weights2=1/len(parallelResults2)
    return parallelResults2,weights2,index2

def allKindsOfPattern(ringWidth,diskSize):
    global splitRate
    splitRate=[2,3,4,5,6,7,8]
    #global splitRate=[2,3]
    global splitNum
    splitNum=20
    ringTemplate =[]
    index=[]
    for i in xrange(0,len(diskSize)):
        ring=np.zeros((diskSize[i]*2+1,diskSize[i]*2+1),dtype=np.uint8)
        #画圆方案一：调用函数MidpointCircle
        #for j in xrange(1,ringWidth+1):
            #ring = MidpointCircle(ring,diskSize[i]-j+1,diskSize[i],diskSize[i],1)
        #画圆方案二：调用opencv.circle()
        cv2.circle(ring,(np.int32(diskSize[i]),np.int32(diskSize[i])),np.int32(diskSize[i]),1)
        ring = cv2.dilate(ring,diskStrelKernel)
        ringTemplateTop=[]
        #print 'i=%d'%i
        #print ring
        for j in xrange(0,len(splitRate)):
            ringTop=cp.deepcopy(ring)
            ringTop[0:round(ringTop.shape[0]*splitRate[j]/splitNum):,:] = 0
            ringTemplateBottom=[]
            #print 'i=%d,j=%d'%(i,j)
            #print ringTop
            #print ring
            #cv2.imshow('test',ringTop*255)
            #cv2.waitKey(3000)
            for k in xrange(0,len(splitRate)):
                ringBottom=cp.deepcopy(ringTop)
                ringBottom[round(ringBottom.shape[0]*(splitNum-splitRate[k])/20):,:]=0
                #print 'i=%d,j=%d,k=%d'%(i,j,k)
                #cv2.imshow('test',ringBottom*255)
                #cv2.waitKey(1000)
                ringTemplate.append(ringBottom)
                index.append([i,j,k])
        #       ringTemplateBottom.append(ringBottom)
#       ringTemplateTop.append(ringTemplateBottom)
#   ringTemplate.append(ringTemplateTop)
    return ringTemplate,index

